In [149]:
### start_imports ###
import os
import re
import json
from utils import *
### end_imports ###
from pyperclip import copy

In [150]:
### start_miscellaneous ###
def read_ipynb(input_ipynb_file):
    # Check if the input file is a Jupyter Notebook
    
    # Open the input Jupyter Notebook file
    notebook = open(input_ipynb_file,encoding="utf-8")
    
    # Read its content in the json format
    notebook_content = json.load(notebook)

    # Only extract the source code snippet from each cell in the input Jupyter Notebook
    source_code_snippets = [cell['source'] for cell in notebook_content['cells']]
    text = ""
    for cell in source_code_snippets:
        for line in cell:
            text+= line
        text+= "\n"

    return text
### end_miscellaneous ###

In [151]:
#### start_function ### function_name = ipynb_2_py
### start_function_vars ###
### start_funcion_defined_vars ###
save_file_name =  False #'./BIU.py'
overwrite = True
### end_funcion_defined_vars ###
text_file_name  =    './checar_rapeel.ipynb'  #"./Previsor_teste3.ipynb"
### end_function_vars ###


if not save_file_name:
    dir,file = os.path.split(os.path.abspath(text_file_name))
    file = file.split(".")[0]
    save_file_name = os.path.join(dir,f"{file}.py")

In [152]:
text = read_ipynb(text_file_name)
#text = re.sub(r" *\n", '\\n', text, 0, re.MULTILINE)
text = re.sub(r"###\n###", '###\\n\\n###', text, 0, re.MULTILINE)
#text = re.sub(r" {2,}", ' ', text, 0, re.MULTILINE)
text = re.sub(r"display.*", '', text, 0, re.MULTILINE)
text = re.sub(r"\"\"\" ### include_on_script ###(.*?)\"\"\"", "\\1", text, 0, re.MULTILINE | re.DOTALL)


functions = re.findall(r"#### start_function ###(.*?)\n### end_function ###",
                    text, re.MULTILINE | re.DOTALL)

imports = re.search(r"### start_imports ###\n(.*)\n### end_imports ###",
                        text, re.MULTILINE | re.DOTALL|re.UNICODE).group(1)

miscellaneous = re.search(r"### start_miscellaneous ###\n(.*)\n### end_miscellaneous ###",
                        text, re.MULTILINE | re.DOTALL|re.UNICODE).group(1)

py_code = f"{imports}\n\n{miscellaneous}\n\n"
functions_code = {}

test_function = ""


for function in functions:

    function_vars_block = re.search(r"### start_function_vars ###\n(.*)\n### end_function_vars ###",
                        function, re.MULTILINE | re.DOTALL).group(1)
    dummy = ""

    try:
        function_defined_vars_code = re.search(r"### start_funcion_defined_vars ###\n(.*)\n### end_funcion_defined_vars ###",
                            function, re.MULTILINE | re.DOTALL).group(1)



        for line in function_defined_vars_code.split("\n"):
            if line:
                var,value = line.replace(" ",'').split("=")
                dummy += f"{var} = {value}, "
    except AttributeError:
        pass
    function_defined_vars = dummy

    dummy = ""
    try:
        function_vars_code = re.search(r"### end_funcion_defined_vars ###\n(.*)\n### end_function_vars ###",
                            function, re.MULTILINE | re.DOTALL).group(1)


        for line in function_vars_code.split("\n"):
            if line:
                var,*value = line.replace(" ",'').split("=")
                dummy += f"{var}, "
    except AttributeError:
        pass
    function_vars = dummy
    function_arg = function_vars +  function_defined_vars
    function_name,code = re.findall(r"function_name = (.*?)\n.*### end_function_vars ###\n(.*)",
                        function, re.MULTILINE | re.DOTALL)[0]

    #code = re.sub(r"\"\"\" ### include_on_script ###\n(.*)\"\"\"", "\\1", code, 0, re.MULTILINE | re.DOTALL)
    dummy = ""
    for line in code.split("\n"):
        dummy += " "*4 + line + '\n'
    code = dummy
    function_code = f"### start {function_name} ###\ndef {function_name}({function_arg}):\n{dummy}### end {function_name} ###\n"
    functions_code[function_name] = function_code
    py_code+= function_code
    test_function += (re.findall(r"### start_funcion_defined_vars ###\n(.*)### end_funcion_defined_vars ###",function,re.MULTILINE | re.DOTALL)[0] + 
            re.findall(r"### end_funcion_defined_vars ###\n(.*)### end_function_vars ###",function,re.MULTILINE | re.DOTALL)[0] + "\n" +
            f"{function_name}({function_arg})")


py_code+= "\nif __name__ == '__main__':\n"
for line in test_function.split("\n"):
    py_code += "\t" + line + "\n"

if not os.path.exists(save_file_name) or overwrite:
    print("arquivo criado...")
    write_file(py_code,save_file_name)
else:
    existing_file = read_file(save_file_name)
    for function_name in functions_code:
        if len(re.findall(rf"### start {function_name} ###\n.*### end {function_name} ###", existing_file,re.MULTILINE | re.DOTALL)) != 1:
            raise
        existing_file = re.sub(f"### start {function_name} ###\n.*### end {function_name} ###", fr"{functions_code[function_name]}",existing_file,0, re.MULTILINE | re.DOTALL)
    print("Arquivo sobrescrito")
    write_file(existing_file,save_file_name)   
### end_function ###

arquivo criado...
